Reading notes and partial solutions to [Data Structures and Algorithms in Python](https://blackwells.co.uk/bookshop/product/9781118290279?gC=f177369a3b&gclid=Cj0KCQjwhJrqBRDZARIsALhp1WTBIyoxeQGXedlVy80vsglvFbNkVf7jTP0Z0zXEIP87lfqbtb4_diYaAr8dEALw_wcB).

In [1]:
import random
from matplotlib import pyplot as plt
%matplotlib inline
import math
from datetime import datetime
import time
import numpy as np

In [1]:
import sys
sys.path.append("..")
from chapter7.positional_list import PositionalList

# Priority Queues

## Priority Queue ADT

In [2]:
class PriorityQueueBase:
    """Abstract base class for a priority queue."""
    
    class _Item:
        """Composite to store priority queue items."""
        __slots__ = "_key", "_value"
        
        def __init__(self, k, v):
            self._key = k
            self._value = v
        
        def __lt__(self, other):
            return self._key < other._key
    
    def is_empty(self):
        """Return True if the priority queue is empty."""
        return len(self) == 0

### Unsorted Priority Queue

In [3]:
class UnsortedPriorityQueue(PriorityQueueBase):
    """Priority queue implemented with unsorted list."""
    
    def _find_min(self):
        """Utility function to find item with minimum key."""
        if self.is_empty():
            raise Empty("Priority queue is empty")
        small = self._data.first()
        walk = self._data.after(small)
        while walk is not None:
            if walk.element() < small.element():
                small = walk
            walk = self._data.after(walk)
        return small
    
    def __init__(self):
        """Create a new empty Priority Queue."""
        self._data = PositionalList()
    
    def __len__(self):
        """Return the number of items in the priority queue."""
        return len(self._data)
    
    def add(self, key, value):
        """Add a key-value pair."""
        self._data.add_last(self._Item(key, value)) # add_last() method of PositionalList
    
    def min(self):
        """Return but do not remove (k, v) tuple with minimum key."""
        p = self._find_min()
        item = p.element()
        return (item._key, item._value)
    
    def remove_min(self):
        """Remove and return (k, v) tuple with minimum key."""
        p = self._find_min()
        item = self._data.delete(p)
        return (item._key, item._value)
    

### Sorted Priority Queue

In [ ]:
class SortedPriorityQueue(PriorityQueueBase):
    """Priority queue implemented with a sorted list."""
    
    def __init__(self):
        """Create a new empty Priority Queue."""
        self._data = PositionalList()
    
    def __len__(self):
        """Return the number of items in the priority queue."""
        return len(self._data)
    
    def add(self, key, value):
        """Add a key-value pair."""
        newest = self._Item(key, value)
        walk = self._data.last() # walk backward to look for smaller key
        while walk is not None and newest < walk.element():
            walk = self._data.before(walk)
        if walk is None: # new key is smallest
            self._data.add_first(newest)
        else:
            self._data.add_after(walk, newest) # newest goes after walk
            
    def min(self):
        """Return but do not remove (k, v) tuple with minimum key."""
        if self.is_empty():
            raise Empty("Priority queue is empty")
        item = self._data.first()
        return (item._key, item._value)
    
    def remove_min(self):
        """Remove and return (k, v) tuple with minimum key."""
        if self.is_empty():
            raise Empty("Priority queue is empty.")
        item = self._data.delete(self._data.first())
        return (item._key, item._value)

## Heaps

### Heap Priority Queue

In [4]:
class HeapPriorityQueue(PriorityQueueBase):
    """Priority queue implemented with a binary heap."""
    
    def _parent(self, j):
        return (j-1)//2
    
    def _left(self, j):
        return 2*j + 1
    
    def _right(self, j):
        return 2*j + 2
    
    def _has_left(self, j):
        return self._left(j) < len(self._data)
    
    def _has_right(self, j):
        return self._right(j) < len(self._data)
    
    def _swap(self, i, j):
        """Swap elements at indices i and j."""
        self._data[i], self._data[j] = self._data[j], self._data[i]
        
    def _upheap(self, j):
        """Up-heap bubbling to adjust position of element at index j to maintain heap order."""
        parent = self._parent(j)
        if j > 0 and self._data[j] < self._data[parent]:
            self._swap(j, parent)
            self._upheap(parent) # recur at parent position
    
    def _downheap(self, j):
        """Down-heap bubbling to adjust positgion of element at index j to maintain heap order."""
        if self._has_left(j):
            left = self._left(j)
            small_child = left # smaller child of left and right
            if self._has_right(j):
                right = self._right(j)
                if self._data[right] < self._data[left]:
                    small_child = right
            if self._data[small_child] < self._data[j]:
                self._swap(j, small_child)
                self._downheap(small_child)
    
    def __init__(self):
        """Create a new empty Priority Queue."""
        self._data = [] # array-based
        
    def __len__(self):
        """Return the number of items in the priority queue."""
        return len(self._data)

    def add(self, key, value):
        """Add a key-value pair to the priority queue."""
        self._data.append(self._Item(key, value)) # add to the end of array
        self._upheap(len(self._data) - 1) # upheap bubbling
    
    def min(self):
        """Return but do not remove (k, v) tuple with minimum key."""
        if self.is_empty():
            raise Empty("Priority queue is empty.")
        item = self._data[0] # item with minimum key is at top of the heap
        return (item._key, item._value)
    
    def remove_min(self):
        """Remove and return (k, v) tuple with minimum key."""
        if self.is_empty():
            raise Empty("Priority queue is empty.")
        self._swap(0, len(self._data) - 1)
        item = self._data.pop()
        self._downheap(0) # downheap bubbling
        return (item._key, item._value)

## Sorting with a Priority Queue

1. Add each element to the priority queue.
2. Keep removing the minimum element until the priority queue is empty.

In [5]:
def pq_sort(C):
    """Sort a collection of elements stored in a positional list."""
    n = len(C)
    P = PriorityQueue()
    for j in range(n):
        element = C.delete(C.first())
        P.add(element, element) # use element as key and value
    for j in range(n):
        (k, v) = P.remove_min()
        C.add_last(v)

### Sorting with Unsorted Priority Queue
If the priority queue is implemented using an unsorted list ($O(1)$ `add` and $O(n)$ `remove_min`), then `pq_sort` is the $O(n^2)$ selection-sort, where the selection of minimums in the unsorted sub-list is $O(n)$.

### Sorting with Sorted Priority Queue
If the priority queue uses a sorted list ($O(n)$ `add` and $O(1)$ `remove_min`), then `pq_sort` is the $O(n^2)$ insertion-sort, where the insertion of next element to the sorted sub-list is $O(n)$.

Best-case runtime for selection-sort is $O(n^2)$, for insertion-sort is $O(n)$ (because it always takes $O(n)$ to find the minimum in a sub-list even if it's sorted, but it only takes $O(1)$ to insert a largest element to an already-sorted sub-list).

### Heap-sort
If the priority queue is implemented using a heap ($O(\log n)$ `add` and `remove_min`), then `pq_sort` is the $O(n\log n)$ heap-sort.

## Adaptable Priority Queues

In [6]:
class AdaptableHeapPriorityQueue(HeapPriorityQueue):
    """A locator-based priority queue implemented with a binary heap."""
    
    class Locator(HeapPriorityQueue._Item):
        """Token for locating an entry of the priority queue."""
        __slots__ = "_index" # add index as additional field
        
        def __init__(self, k, v, j):
            super().__init__(k, v)
            self._index = j
        
    def _swap(self, i, j):
        super()._swap(i, j)
        self._data[i]._index = i # reset locator index after swap
        self._data[j]._index = j

    def _bubble(self, j):
        if j > 0 and self._data[j] < self._data[self._parent(j)]:
            self._upheap(j)
        else:
            self._downheap(j)

    def add(self, key, value):
        """Add a key-value pair."""
        token = self.Locator(key, value, len(self._data)) # initialize locator index
        self._data.append(token)
        self._upheap(len(self._data) - 1)
        return token

    def update(self, loc, newkey, newval):
        """ Update the key and value for the entry identified by Locator loc."""
        j = loc._index
        if not (0 <= j and j < len(self) and self._data[j] is loc):
            raise ValueError("Invalid locator")
        if j == len(self) - 1:
            self._data.pop()
        else:
            self._swap(j, len(self) - 1) # swap item to the last position
            self._data.pop()
            self._bubble(j) # fix items displaced by the swap
        return (loc._key, loc._value)